In [41]:
import importlib
# --------------------
from sklearn.utils import gen_batches
from sklearn.model_selection import GridSearchCV, KFold, TimeSeriesSplit
import numpy as np
from numpy.linalg import norm,inv,matrix_rank
import h5py
import datetime

from Code.SGDPLS import SGDPLS
from Code.OLPLS import OLPLS
from Code.CIPLS import CIPLS
from Code.SIMPLS import SIMPLS
from Code.ISIMPLS import ISIMPLS2 as ISIMPLS
from Code.IPLS import IPLS
# ----------
from sklearn.cross_decomposition import PLSRegression
from pytictoc import TicToc
tim=TicToc()

In [42]:
def gen_slice_month(tim_st,tim_ed,num):
    tim = datetime.datetime.strptime(tim_st, "%Y%m%d")
    date_ed = datetime.datetime.strptime(  tim_ed, "%Y%m%d")
    ind_st = 0
    batches  = []
    trainset = []
    samples  = []
    count = 0
    while (tim < date_ed ):
        Year = tim.strftime("%Y")
        Month = tim.strftime("%m")
        if Month == '12':
            NextYYMM = "{:04d}01".format(int(tim.strftime("%Y"))+1)
        else:
            NextYYMM = Year+"{:02d}".format(int(tim.strftime("%m"))+1)
        tmp = datetime.datetime.strptime(  NextYYMM+"01" , "%Y%m%d")
        if (tmp > date_ed):
            tmp = date_ed
        days = (tmp - tim ).days
        ind_ed = ind_st + days
        batches.append(slice(ind_st,ind_ed))
        trainset.append(slice(ind_ed))
        samples.append(ind_ed)
        # count += 1
        # if (count >= num):
        #     return batches,trainset,samples
        ind_st = ind_ed
        tim += datetime.timedelta(days=days)
    return batches,trainset,samples

In [47]:
tim_st = '20200602'
tim_ed = '20221027'

tmp = h5py.File('./data/TW_PM25.h5', 'r')
X_train, Y_train = tmp['X_train'], tmp['Y_train']
n_train = X_train.shape[0]
Y_train = Y_train[0:n_train]
batches, trainset, samples = gen_slice_month(tim_st,tim_ed,batch_num)

n_train = samples[-1]
train_ind = trainset[-1]

X_train, Y_train = X_train[0:n_train], Y_train[0:n_train]


In [48]:
print(len(batches))
print("n_train: ", n_train)
print("trainset:", trainset)
print("size: X_train={}, Y_train={}".format(X_train.shape, Y_train.shape))

29
n_train:  877
trainset: [slice(None, 29, None), slice(None, 60, None), slice(None, 91, None), slice(None, 121, None), slice(None, 152, None), slice(None, 182, None), slice(None, 213, None), slice(None, 244, None), slice(None, 272, None), slice(None, 303, None), slice(None, 333, None), slice(None, 364, None), slice(None, 394, None), slice(None, 425, None), slice(None, 456, None), slice(None, 486, None), slice(None, 517, None), slice(None, 547, None), slice(None, 578, None), slice(None, 609, None), slice(None, 637, None), slice(None, 668, None), slice(None, 698, None), slice(None, 729, None), slice(None, 759, None), slice(None, 790, None), slice(None, 821, None), slice(None, 851, None), slice(None, 877, None)]
size: X_train=(877, 1800), Y_train=(877, 1800)


In [49]:
def RunExp(method, PLS, case, save):
    tim = TicToc()
    info_list = {}
    if (case==1):
        timlist=np.zeros((2, len(batches)))
        timlist[0] = np.array(samples)
        for i,s in enumerate(batches):
            tim.tic()
            PLS.fit(X_train[s], Y_train[s])
            timlist[1,i] = tim.tocvalue()
    else:
        tim.tic()
        PLS.fit(X_train[train_ind], Y_train[train_ind])
        tim.tocvalue()

    ypred = PLS.predict(X_train[train_ind])
    coef  = PLS.coef_

    if (save==1):
        with h5py.File(f"./Results/PLS2_PM25_{method}.h5", "w") as f:
            f_coef = f.create_dataset('coef', data=coef, maxshape=coef.shape, chunks=True)
            f_ypred = f.create_dataset('ypred', data=ypred, maxshape=ypred.shape, chunks=True)
            if (case==1):
                f_tim = f.create_dataset('timer', data=timlist, maxshape=timlist.shape, chunks=True)


In [50]:
import Code.SIMPLS
importlib.reload(Code.SIMPLS)
from Code.SIMPLS import SIMPLS

save=1
tim.tic()
PLS = SIMPLS(n_components=11)
RunExp('SIMPLS', PLS, 0, save)
print("elapsed time:", tim.tocvalue() )

In [51]:
import Code.ISIMPLS
importlib.reload(Code.ISIMPLS)
from Code.ISIMPLS import ISIMPLS2 as ISIMPLS

save=1
tim.tic()
PLS= ISIMPLS(n_components=11)
RunExp('ISIMPLS', PLS, 1, save)
print("elapsed time:", tim.tocvalue() )

In [55]:
save=1
tim.tic()
PLS= PLSRegression(n_components=10)
RunExp('NIPALS', PLS, 0, save)
print("elapsed time:", tim.tocvalue() )

NameError: name 'tic' is not defined

In [53]:
import Code.OLPLS
importlib.reload(Code.OLPLS)
from Code.OLPLS import OLPLS

save=1
tim.tic()
PLS= OLPLS(n_components=10, mu=1e-8, amnesic=0.99)
RunExp('OLPLS', PLS, 1, save)
print("elapsed time:", tim.tocvalue() )

KeyboardInterrupt: 